In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [2]:
%cd Z:\Project_Top2B\April_2023\regionwise\cerebellum
data = pd.read_csv('Merged_Top2B_cb.csv')
data = data.drop(['Sample_ID'], axis = 1)
data.head(1)

Z:\Project_Top2B\April_2023\regionwise\cerebellum


,Age,cg00010266,cg00018024,cg00018539,cg00019511,cg00029353,cg00030423,cg00034416,cg00034611,cg00041368,...,cg27551895,cg27569265,cg27597926,cg27601855,cg27624319,cg27629898,cg27633533,cg27635069,cg27658967,cg27664120
0,2.0,0.073333,0.050456,0.868932,0.881951,0.054426,0.069267,0.057713,0.854109,0.91927,...,0.158858,0.155456,0.081959,0.168745,0.159493,0.8402,0.918704,0.426576,0.066982,0.856399


In [3]:
# Replace '' values with NaN
data = data.replace('', np.nan)

# Loop over each column and replace NaN values with column means
for col in data.columns:
    # Convert the column to numeric data type
    data[col] = pd.to_numeric(data[col])
    
    # Calculate the mean of the column
    col_mean = data[col].mean()
    
    # Replace NaN values with the column mean
    data[col].fillna(col_mean, inplace=True)

data.to_csv("Merged_Top2B_Mean_cb.csv", index = False)
data.head(2)

,Age,cg00010266,cg00018024,cg00018539,cg00019511,cg00029353,cg00030423,cg00034416,cg00034611,cg00041368,...,cg27551895,cg27569265,cg27597926,cg27601855,cg27624319,cg27629898,cg27633533,cg27635069,cg27658967,cg27664120
0,2.0,0.073333,0.050456,0.868932,0.881951,0.054426,0.069267,0.057713,0.854109,0.919270,...,0.158858,0.155456,0.081959,0.168745,0.159493,0.840200,0.918704,0.426576,0.066982,0.856399
1,6.0,0.063563,0.071539,0.837630,0.871659,0.067022,0.075092,0.071654,0.708397,0.903124,...,0.205979,0.135422,0.064384,0.170143,0.093049,0.827394,0.882720,0.425566,0.071315,0.851720


In [4]:
X = data.drop(['Age'], axis = 1)
y = data['Age']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 

In [5]:
rf_all_features = RandomForestRegressor(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestRegressor(random_state=1)

accuracy_score(y_test, rf_all_features.predict(X_test))

In [6]:
rfc = RandomForestRegressor(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100
Iteration: 35 / 100
Iteration: 36 / 100
Iteration: 37 / 100
Iteration: 38 / 100
Iteration: 39 / 100
Iteration: 40 / 100
Iteration: 41 / 100
Iteration: 42 / 100
Iteration: 43 / 100
Iteration: 44 / 100
Iteration: 45 / 100
Iteration: 46 / 100
Iteration: 47 / 100
Iteration: 48 / 100
Iteration: 49 / 100
Iteration: 50 / 100
Iteration

BorutaPy(estimator=RandomForestRegressor(n_estimators=125,
                                         random_state=RandomState(MT19937) at 0x20D90F89540),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x20D90F89540, verbose=1)

In [7]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  49


In [8]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [9]:
selected_rf_features.to_csv ('CpG_BorutaPy_onAll_cb.csv', index = False)